# Tutorial 02: overview of **create** stages

Let's do a quick run-through of stages we might apply to a patch before implanting in an image.

All of these stages can input a tensor containing a batch of patches, or a dictionary of tensors for the multi-patch case.

In [ ]:
import electricmayhem.whitebox as em

Loading a test image...

In [ ]:
flower = em.load_to_tensor("data/flower2.png")
em.plot(flower)

## Resizing

Depending on how you're implanting the patch you may want to resize it first:

In [ ]:
resize = em.PatchResizer((20,20))

In [ ]:
resized, _ = resize(flower.unsqueeze(0))
em.plot(resized)

## Stacking

For use cases where a black-and-white patch is being implanted in an RGB images.

In [ ]:
stacker = em.PatchStacker(3)

In [ ]:
stacked, _ = stacker(flower[:1,:,:].unsqueeze(0))
print(stacked.shape)
em.plot(stacked)

## Discouraging patches from becoming inversions

It's easy for decoy patches to wind up turning into pictures of the target class. Some tricks we can apply that modify the patch if we want to avoid that:

Make a large patch by tiling a small patch:

In [ ]:
tiler = em.PatchTiler((640,480))

In [ ]:
tiled, _ = tiler(flower.unsqueeze(0))
em.plot(tiled)

Filter out low-frequency Fourier components during training steps:

In [ ]:
highpass = em.HighPassFilter(limit=25)

In [ ]:
filtered, _ = highpass(flower.unsqueeze(0))
em.plot(filtered)

Randomly translate the patch with toroidal boundary conditions during train steps:

In [ ]:
scroller = em.PatchScroller()

In [ ]:
scrolled, _ = scroller(flower.unsqueeze(0))
em.plot(scrolled)

## Adding noise during training

The paper *Frequency Domain Model Augmentation for Adversarial Attack* by Long *et al* (2022) only discusses digital attacks, but it's straightforward to apply to patches as well:

In [ ]:
ssa = em.SpectrumSimulationAttack()

In [ ]:
noised, _ = ssa(flower.unsqueeze(0))
em.plot(noised)

## Soft-proofing

Off-the-shelf color management tools aren't differentiable but we can still apply them during evaluation steps. If you build or download an ICC profile for your printer, it's straightforward to simulate what the patch would look like printed out. This can be helpful for digitally evaluating methods like NPS loss.

In [ ]:
proofer = em.SoftProofer("data/profile.icc")

In [ ]:
proofed, _ = proofer(flower.unsqueeze(0), evaluate=True)
em.plot(proofed)